<a href="https://colab.research.google.com/github/Auroraaa17/RAG-pipeline/blob/main/RAG-pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers einops accelerate langchain bitsandbytes
!pip install sentence_transformers
!pip install llama_index
!pip install llama-index-llms-huggingface
!pip install langchain_community
!pip install accelerate

In [ ]:
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.prompts.prompts import SimpleInputPrompt
from llama_index.core import VectorStoreIndex,SimpleDirectoryReader,ServiceContext

In [ ]:
Documents = SimpleDirectoryReader("/content/arxiv_papers").load_data()
Documents

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains.llm import LLMChain


system_prompt = """
You are a friendly chatbot assistant that gives structured output.
Your role is to arrange given task in this structure.
### instruction:
{instruction}
Output:
"""
task_prompt_template = PromptTemplate(
    input_variables=["instruction"], template=system_prompt
)

In [ ]:

from huggingface_hub import interpreter_login

interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: ··········
Add token as git credential? (Y/n) Y
Token is valid (permission: read).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
 !pip install -i https://pypi.org/simple/ bitsandbytes

In [ ]:
from llama_index.llms.huggingface import HuggingFaceLLM
from transformers import BitsAndBytesConfig
from transformers import AutoTokenizer, PhiForCausalLM

#from accelerate import disk_offload
#from transformers import AutoModelForCausalLM
import torch

#model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf", offload_folder="save_folder", torch_dtype=torch.float16, low_cpu_mem_usage = True).cpu()
#from accelerate import disk_offload
#disk_offload(model=model,offload_dir= "offload_directory" )


quantization_config = BitsAndBytesConfig(load_in_8bit=True, llm_int8_threshold=200.0)
#model = AutoModelForCausalLM.from_pretrained("HuggingFaceH4/zephyr-7b-beta", torch_dtype=torch.float16, low_cpu_mem_usage = True).cpu()
#from accelerate import disk_offload
#disk_offload(model=model, offload_dir="offload")

llm = HuggingFaceLLM(
    context_window=5000,
    max_new_tokens=256,
    generate_kwargs={"temperature" : 0.0, "do_sample": False},
    system_prompt=system_prompt,

    tokenizer_name = "microsoft/phi-2",
    model_name = "microsoft/phi-2",
    device_map = "auto",
    #quantization_config=quantization_config,
)
    #model_kwargs ={"torch_dtype" : torch.float16, "load_in_8bit" : True}
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-2", trust_remote_code=True)  # execs code downloaded from hf hub
tokenizer.pad_token = tokenizer.eos_token






/usr/local/lib/python3.10/dist-packages/accelerate/utils/modeling.py:1363: UserWarning: Current model requires 16779264 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
pip install llama-index-embeddings-langchain


In [ ]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.core import ServiceContext
from llama_index.embeddings.langchain import LangchainEmbedding

In [ ]:
pip install -U sentence-transformers


In [ ]:
embed_model = LangchainEmbedding(
    HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
)

In [ ]:
from llama_index.core import Settings

# maximum input size to the LLM
Settings.context_window = 4096

# number of tokens reserved for text generation.
Settings.num_output = 256

In [ ]:

from llama_index.core import VectorStoreIndex
index = VectorStoreIndex.from_documents(
    Documents, embed_model=embed_model)


query_engine = index.as_query_engine(llm=llm)

In [ ]:
response = query_engine.query("What is CRUD?")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
print(response)



CRUD-RAG: A Comprehensive Chinese Benchmark for Retrieval-Augmented Generation of Large Language Models 111:3
Fig. 1. We have classified the application scenarios of RAG into four primary aspects: Create, Read, Update,
and Delete. The figure provides an illustrative example for each category, showcasing the wide-ranging
potential of RAG technology.
we can use the CRUD framework to classify the RAG systems’ application scenarios. As shown in
Figure 1, each CRUD category demonstrates different capabilities of the RAG system:
•In "CREATE ", the system improves the input text by adding relevant information from
external sources, making creative outputs such as poetry, stories, or code.
•In "READ ", the system uses external knowledge retrieval to answer questions, solving
problems in question-answering, dialogue, and reasoning, and increasing understanding of
the input text.
•In "UPDATE ", the system fixes errors in the input text using retrieved content, correcting
spelling, grammar or f